# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,edinburgh of the seven seas,-37.0676,-12.3116,10.03,60,84,8.45,SH,1722983648
1,nar'yan-mar,67.6713,53.0870,5.43,85,20,3.33,RU,1722983649
2,kenai,60.5544,-151.2583,16.68,93,100,3.09,US,1722983651
3,puerto natales,-51.7236,-72.4875,-1.75,93,75,0.00,CL,1722983541
4,lihue,21.9789,-159.3672,28.19,71,75,6.69,US,1722983653


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity"]
)

# Display the map
city_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
conditions_df = city_data_df[(city_data_df["Max Temp"] >= 5) & (city_data_df["Max Temp"] <= 40) &
                            (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <=90)]
# Drop any rows with null values
Ideal = conditions_df.dropna()

# Display sample data
Ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,edinburgh of the seven seas,-37.0676,-12.3116,10.03,60,84,8.45,SH,1722983648
1,nar'yan-mar,67.6713,53.0870,5.43,85,20,3.33,RU,1722983649
4,lihue,21.9789,-159.3672,28.19,71,75,6.69,US,1722983653
7,los zacatones,22.9500,-102.0500,27.51,26,21,4.30,MX,1722983656
8,uzgen,40.7699,73.3007,22.16,40,1,2.62,KG,1722983658
...,...,...,...,...,...,...,...,...,...
586,parang,7.3704,124.2697,24.91,88,100,1.85,PH,1722984355
587,erenhot,43.6475,111.9767,24.42,39,1,2.35,CN,1722984357
588,bukama,-9.2000,25.8500,21.71,22,2,0.62,CD,1722984358


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal[[
                  'City',
                  'Country',
                  'Lat',
                  'Lng',
                  'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,60,
1,nar'yan-mar,RU,67.6713,53.0870,85,
4,lihue,US,21.9789,-159.3672,71,
7,los zacatones,MX,22.9500,-102.0500,26,
8,uzgen,KG,40.7699,73.3007,40,
...,...,...,...,...,...,...
586,parang,PH,7.3704,124.2697,88,
587,erenhot,CN,43.6475,111.9767,39,
588,bukama,CD,-9.2000,25.8500,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
#Latitude = ?
#Longitude = ?
categories="accommodation.hotel"
limit = 10
#conditions=
#filters= f"circle:{longitude}{latitude}"
#bias = f"proximity:{longitude}{latitude}"
radius = 10000
params = {
    "categories": categories,
    "limit":limit,
    "apiKey": geoapify_key
}

#    "filter": filters,"bias": bias,

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #latitude = hotel_df.loc[index, "Lat"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    #longitude = hotel_df.loc[index, "Lng"]
    


    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: Гостиница "Агат"
lihue - nearest hotel: Kauai Palms
los zacatones - nearest hotel: No hotel found
uzgen - nearest hotel: Кунрат
stanley - nearest hotel: Hotel 52
akureyri - nearest hotel: Hótel Norðurland
tafuna - nearest hotel: Tradewinds
klyuchi - nearest hotel: No hotel found
wloszczowa - nearest hotel: Rezydencja TARy
magaramkent - nearest hotel: Самур
nago - nearest hotel: グリーンリッチホテル
adamstown - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
iqaluit - nearest hotel: Frobisher Inn
bekovo - nearest hotel: No hotel found
tianshui - nearest hotel: 丽都假日酒店
bonthe - nearest hotel: No hotel found
vilyuysk - nearest hotel: Мария
laojunmiao - nearest hotel: No hotel found
camabatela - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
carnarvon - nearest hotel: No hotel found
mount isa - n

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,60,No hotel found
1,nar'yan-mar,RU,67.6713,53.0870,85,"Гостиница ""Агат"""
4,lihue,US,21.9789,-159.3672,71,Kauai Palms
7,los zacatones,MX,22.9500,-102.0500,26,No hotel found
8,uzgen,KG,40.7699,73.3007,40,Кунрат


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_data_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo= True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Lng", "Lat", "City", "Humidity","Hotel Name", "Country"]
)

# Display the map
hotel_data_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)